# Named Entity Recognition

## Set variables

In [1]:
file_name = "BiodivNER_AlBERT_ver1.ipynb"
batch_size = 4
dataset = "BiodivNER"
data_directory = "../Datasets/NER/COPIOUS-txt/"
model_src = "albert-base-v2"
model_name = "biodivner_albert-base-v2" # (wandb project name)

## Install and import necessary libraries

In [2]:
# !pip install transformers
# !pip install datasets
# !pip install seqeval
# !pip install accelerate -U

In [3]:
import os
import shutil
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Set up Weights and Biases

In [4]:
os.environ["WANDB_NOTEBOOK_NAME"] = file_name

In [5]:
# !pip install wandb
import wandb
wandb.login()

wandb: Currently logged in as: shannen (shannen-team). Use `wandb login --relogin` to force relogin


True

Set up experiment and hyperparameters

In [6]:
wandb.init(
    project=model_name,
    config={
        "batch_size": batch_size,
        "dataset": dataset,
    },
)

## Convert the dataset to CoNLL2003 format

In [7]:
# def read_CoNLL2003_format(filename, idx=3):
#     """Read file in CoNLL-2003 shared task format"""

#     # read file
#     lines =  open(filename, encoding="utf-8").read().strip()

#     # find sentence-like boundaries
#     lines = lines.split("\n\n")

#      # split on newlines
#     lines = [line.split("\n") for line in lines]

#     # get tokens
#     tokens = [[l.split()[0] for l in line] for line in lines]

#     # get labels/tags
#     labels = [[l.split()[idx] for l in line] for line in lines]

#     #convert to df
#     data= {'tokens': tokens, 'labels': labels}
#     df=pd.DataFrame(data=data)

#     return df

In [8]:
# def flatten(l):
#     return [item for sublist in l for item in sublist]

In [9]:
# DATADIR = data_directory

# def get_data(trainfile=DATADIR + "train.txt",
#              devfile=DATADIR + "dev.txt",
#              testfile=DATADIR + "test.txt"):

#     train = read_CoNLL2003_format(trainfile, idx=3)
#     dev = read_CoNLL2003_format(devfile, idx=3)
#     print("Train data: %d sentences, %d tokens"%(len(train),len(flatten(train.tokens))))

#     print("Dev data: %d sentences, %d tokens"%(len(dev),len(flatten(dev.tokens))))

#     test = read_CoNLL2003_format(testfile, idx=3)
#     print("Test data: %d sentences, %d tokens"%(len(test),len(flatten(test.tokens))))

#     return train, test, dev

In [10]:
# train, test, dev = get_data()

# train_dataset = Dataset.from_pandas(train)
# dev_dataset = Dataset.from_pandas(dev)
# test_dataset = Dataset.from_pandas(test)

In [11]:
# print(train)

In [12]:
# print(train_dataset)

In [13]:
root_data_dir = "../Datasets/NER/BiodivNER/"

biodivner_dataset = "train"
train_csv_file_path = "train.csv"
val_csv_file_path = "dev.csv"
test_csv_file_path = "test.csv"

In [14]:
def loadData(csv_file_path):
  dataset_path = os.path.join(root_data_dir, csv_file_path)
  data = pd.read_csv(dataset_path, encoding="latin1")
  data = data.fillna(method="ffill")
  return data

In [15]:
data = loadData(train_csv_file_path)
val_data = loadData(val_csv_file_path)
test_data = loadData(test_csv_file_path)

/tmp/ipykernel_1640852/3117403119.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")
/tmp/ipykernel_1640852/3117403119.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")
/tmp/ipykernel_1640852/3117403119.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill")


In [16]:
data.head(105)

,Sentence #,Word,Tag
0,Sentence: 0,Samplenr,O
1,Sentence: 0,Seedlingnr,O
2,Sentence: 0,Plot,O
3,Sentence: 0,Record,O
4,Sentence: 0,Date,O
...,...,...,...
100,Sentence: 1,",",O
101,Sentence: 1,frames,O
102,Sentence: 1,",",O
103,Sentence: 1,landing,O


In [17]:
import re

def convert(orig):

    df = pd.DataFrame(orig)

    # Extract the integer from 'Sentence #'
    df['Sentence #'] = df['Sentence #'].apply(lambda x: int(re.search(r'\d+', x).group()))

    # Add a column representing the original order
    df['Original Order'] = range(len(df))

    # Group by 'Sentence #' and aggregate 'Word' and 'Tag' into lists
    grouped = df.groupby('Sentence #').agg({'Word': list, 'Tag': list, 'Original Order': 'first'}).reset_index()

    # Sort the DataFrame based on the original order
    grouped = grouped.sort_values(by='Original Order').drop('Original Order', axis=1)

    grouped = grouped.rename(columns={'Word': 'tokens'})
    grouped = grouped.rename(columns={'Tag': 'labels'})
    grouped = grouped.drop('Sentence #', axis=1)
    # print(grouped)
    return grouped

# print(grouped)


In [18]:
train_df = convert(data)
val_df = convert(val_data)
test_df = convert(test_data)

In [19]:
for i in range(len(train_df)):
    # print(len(train_df['tokens'][i]), len(train_df['labels'][i]))
    if len(train_df['tokens'][i]) != len(train_df['labels'][i]):
        print("lol")
# print(train_df['tokens'])

In [20]:
train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

In [21]:
print(train_dataset)

Dataset({
    features: ['tokens', 'labels'],
    num_rows: 1918
})


## Tokenize the dataset

In [22]:
label_list = ['B-Phenomena', 'I-Phenomena', 'B-Quality', 'I-Quality', 'B-Location', 'I-Location', 'B-Environment', 'I-Environment', 'B-Matter', 'I-Matter', 'B-Organism', 'I-Organism', 'O']
label2id = {k: v for v, k in enumerate(label_list)}
id2label = {v: k for v, k in enumerate(label_list)}
print(label2id)
print(id2label)

{'B-Phenomena': 0, 'I-Phenomena': 1, 'B-Quality': 2, 'I-Quality': 3, 'B-Location': 4, 'I-Location': 5, 'B-Environment': 6, 'I-Environment': 7, 'B-Matter': 8, 'I-Matter': 9, 'B-Organism': 10, 'I-Organism': 11, 'O': 12}
{0: 'B-Phenomena', 1: 'I-Phenomena', 2: 'B-Quality', 3: 'I-Quality', 4: 'B-Location', 5: 'I-Location', 6: 'B-Environment', 7: 'I-Environment', 8: 'B-Matter', 9: 'I-Matter', 10: 'B-Organism', 11: 'I-Organism', 12: 'O'}


In [23]:
task = "ner"
model_checkpoint = model_src

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json: 100%|██████████| 684/684 [00:00<00:00, 1.09MB/s]
spiece.model: 100%|██████████| 760k/760k [00:00<00:00, 1.61MB/s]
tokenizer.json: 100%|██████████| 1.31M/1.31M [00:01<00:00, 1.29MB/s]


In [24]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), max_length= 512, truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label2id[label[word_idx]])
            else:
                label_ids.append(label2id[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
dev_tokenized_datasets = dev_dataset.map(tokenize_and_align_labels, batched=True)
test_tokenized_datasets = test_dataset.map(tokenize_and_align_labels, batched=True)

Map: 100%|██████████| 240/240 [00:00<00:00, 1071.71 examples/s]


In [25]:
print(train_tokenized_datasets)
print(len(train_tokenized_datasets['tokens'][0]))
print(len(train_tokenized_datasets['labels'][0]))

Dataset({
    features: ['tokens', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1918
})
97
175


## Finetuning the model to the dataset

In [26]:
model =  AutoModelForTokenClassification.from_pretrained(model_checkpoint,id2label=id2label, label2id=label2id)

args = TrainingArguments(
    f"test-{task}",
    per_device_train_batch_size=batch_size,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    evaluation_strategy = "epoch",
    num_train_epochs=3
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {"precision": results["overall_precision"], "recall": results["overall_recall"], "f1": results["overall_f1"], "accuracy": results["overall_accuracy"]}

trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=dev_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

model.safetensors: 100%|██████████| 47.4M/47.4M [00:03<00:00, 13.1MB/s]
Some weights of AlbertForTokenClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1640852/547268042.py:16: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


Clear model directory if it hasn't been cleared yet.

In [27]:
directory_path = "./ner.model"

if os.path.exists(directory_path):
    try:
        # Remove the directory
        shutil.rmtree(directory_path)
        print(f"Directory '{directory_path}' removed successfully.")
    except Exception as e:
        print(f"Error removing directory '{directory_path}': {e}")
else:
    print(f"Directory '{directory_path}' does not exist.")

Directory './ner.model' removed successfully.


In [28]:
trainer.train()
trainer.evaluate()
trainer.save_model('ner.model')

You're using a AlbertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.260500,0.192160,0.568082,0.347048,0.430872,0.938545
2,0.176300,0.171880,0.563367,0.544723,0.553888,0.945318
3,0.126400,0.130588,0.675918,0.675313,0.675615,0.956316


/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


## Making an inference with the finetuned model

In [29]:
from transformers import pipeline

Change model_checkpoint as needed

In [30]:
model_checkpoint = "./ner.model"

# artifact = wandb.use_artifact("electra-small-discriminator:latest")
# model_checkpoint = artifact.download()

In [31]:
token_classifier = pipeline("token-classification", model=model_checkpoint, aggregation_strategy="first")

In [32]:
token_classifier("The cbbL form IA and IC genes associated with carbon fixation were analyzed to gain insight into metabolic potential of chemolithoautotrophs in three types of coastal ecosystems which had a very different salt load and .")

/home/nlpbiodiv2023/.local/lib/python3.10/site-packages/transformers/pipelines/token_classification.py:393: UserWarning: Tokenizer does not support real words, using fallback heuristic
  warnings.warn(


[{'entity_group': 'Matter',
  'score': 0.95551485,
  'word': 'carbon',
  'start': 46,
  'end': 52},
 {'entity_group': 'Environment',
  'score': 0.7179568,
  'word': 'coastalecosystems',
  'start': 159,
  'end': 177},
 {'entity_group': 'Matter',
  'score': 0.90908253,
  'word': 'salt',
  'start': 205,
  'end': 209}]

## Get precision, f1-score, and recall for each entity group

In [33]:
predictions, labels, _ = trainer.predict(test_tokenized_datasets)
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'Environment': {'precision': 0.6417910447761194,
  'recall': 0.6861702127659575,
  'f1': 0.6632390745501285,
  'number': 188},
 'Location': {'precision': 0.44871794871794873,
  'recall': 0.49295774647887325,
  'f1': 0.46979865771812085,
  'number': 71},
 'Matter': {'precision': 0.6950354609929078,
  'recall': 0.49246231155778897,
  'f1': 0.5764705882352941,
  'number': 199},
 'Organism': {'precision': 0.7565392354124748,
  'recall': 0.7673469387755102,
  'f1': 0.7619047619047619,
  'number': 490},
 'Phenomena': {'precision': 0.8214285714285714,
  'recall': 0.6133333333333333,
  'f1': 0.7022900763358778,
  'number': 75},
 'Quality': {'precision': 0.7849462365591398,
  'recall': 0.7572614107883817,
  'f1': 0.7708553326293558,
  'number': 482},
 'overall_precision': 0.7294853963838664,
 'overall_recall': 0.6970099667774087,
 'overall_f1': 0.7128780156303093,
 'overall_accuracy': 0.9433206106870229}

In [34]:
artifact = wandb.Artifact(name=model_name, type="model")
artifact.add_dir(local_path="./ner.model")  # Add dataset directory to artifact
wandb.log(results, commit=True)
wandb.log_artifact(artifact)  # Logs the artifact version "my_data:v0"

wandb: Adding directory to artifact (./ner.model)... 

Done. 0.6s


<Artifact biodivner_albert-base-v2>

In [35]:
wandb.finish()

eval/accuracy,▁▄██
eval/f1,▁▅██
eval/loss,█▆▁▁
eval/precision,▁▁██
eval/recall,▁▅██
eval/runtime,▁▆█▅
eval/samples_per_second,█▃▁▃
eval/steps_per_second,█▃▁▃
overall_accuracy,▁
overall_f1,▁
overall_precision,▁
